In [ ]:
import numpy as np
import os
from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [2]:
#spec = model_spec.get('average_word_vec')
spec = model_spec.get('mobilebert_classifier')
#spec.num_words = 2000
#spec.seq_len = 20
#spec.wordvec_dim = 7
spec.dropout_rate = 0.2
spec.learning_rate = 0.0001

In [4]:
data = DataLoader.from_csv(
    filename="/tmp/spamDetection/cleanuped_spam.csv",
    text_column='comments',
    label_column='label',
    model_spec=spec,
    delimiter=',',
    shuffle=True,
    is_training=True)

train_data, test_data = data.split(0.9)

In [ ]:
# Build the model
# model = text_classifier.create(train_data, model_spec=spec, epochs=50, validation_data=test_data)
model = text_classifier.create(train_data, model_spec=spec, epochs=16)

In [ ]:
loss, acc = model.evaluate(test_data)

In [ ]:
# Save model
export_dir = '/tmp/saved_model/spam'
#model.export(export_dir=export_dir, export_format=ExportFormat.SAVED_MODEL)

In [ ]:
# Save tflite file
model.export(export_dir)

In [ ]:
# Save vocabulary
model.export(export_dir, export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])

In [ ]:
# This will export to TFLite format with the model only. 
# if you see a .json file in this directory, it is NOT the JSON model for TFJS
# See below for how to generate that.
# Please note that if you run this cell to create the tflite model then the 
# export to TFJS will fail. You'll need to rerun the model training first
model.export(export_dir='/tmp/saved_model/spam')
# If you want the labels and the vocab, for example for iOS, you can use this
model.export(export_dir='/tmp/saved_model/spam/', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])

# You can find your files in colab by clicking the 'folder' tab to the left of
# this code window, and then navigating 'up' a directory to find the root
# directory listing -- and from there you should see /mm_spam/

In [ ]:
# Use this section for export to TFJS
# Please note that if you run the above cell to create the tflite model then the 
# export to TFJS will fail. You'll need to rerun the model training first
model.export(export_dir="/tmp/saved_model/tfjs/", export_format=[ExportFormat.TFJS, ExportFormat.LABEL, ExportFormat.VOCAB])